# Sesión 13 — Transformers I: Secuencias y Atención

En esta sesión construiremos la **intuición** y la mecánica base de la atención:
- Tokens → embeddings
- Proyecciones **Q, K, V**
- **Scaled dot-product attention**
- Visualización de pesos de atención

Objetivo: que puedas mirar una matriz de atención y explicar qué está ocurriendo.


In [ ]:
# Imports
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


## 1) Secuencia de ejemplo

Simulamos una secuencia de **N tokens**, cada uno con embedding de dimensión **d_model**.
(Por ahora no hacemos tokenización real; nos enfocamos en el mecanismo.)

In [ ]:
torch.manual_seed(0)
N = 6
d_model = 16
X = torch.randn(N, d_model)
X.shape

## 2) Proyecciones Q, K, V

Para cada token creamos tres proyecciones lineales:
- **Q** (Query): lo que busco
- **K** (Key): lo que ofrezco
- **V** (Value): lo que transmito

In [ ]:
# Matrices de proyección (como si fueran capas lineales sin bias)
Wq = torch.randn(d_model, d_model)
Wk = torch.randn(d_model, d_model)
Wv = torch.randn(d_model, d_model)

Q = X @ Wq
K = X @ Wk
V = X @ Wv

Q.shape, K.shape, V.shape

## 3) Scaled dot-product attention

Computamos similitudes Q·K y normalizamos con softmax:

\[\text{Attention}(Q,K,V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V\]

In [ ]:
d_k = d_model
scores = (Q @ K.T) / (d_k ** 0.5)   # (N x N)
weights = F.softmax(scores, dim=-1)
out = weights @ V                  # (N x d_model)

scores.shape, weights.shape, out.shape

## 4) Visualización: matriz de atención

Cada fila (query) muestra a qué posiciones (keys) presta atención ese token.

In [ ]:
plt.figure()
plt.imshow(weights.detach().cpu())
plt.colorbar()
plt.title('Pesos de atención (self-attention)')
plt.xlabel('Key index (posición atendida)')
plt.ylabel('Query index (token que atiende)')
plt.show()

## 5) Inspección rápida

Veamos, por ejemplo, qué posiciones recibe más peso para el token en la posición 0.

In [ ]:
i = 0
w = weights[i].detach()
top = torch.topk(w, k=3)
top

## 6) Variante opcional: máscara causal (look-ahead)

En modelos autoregresivos (p. ej. GPT), el token i **no** puede mirar posiciones futuras j > i.
Aplicamos una máscara triangular superior antes del softmax.

In [ ]:
mask = torch.triu(torch.ones(N, N), diagonal=1).bool()
scores_masked = scores.clone()
scores_masked[mask] = -1e9
weights_causal = F.softmax(scores_masked, dim=-1)

plt.figure()
plt.imshow(weights_causal.detach().cpu())
plt.colorbar()
plt.title('Pesos de atención con máscara causal')
plt.xlabel('Key index')
plt.ylabel('Query index')
plt.show()

## Preguntas de repaso

1. ¿Qué significa que un token tenga pesos de atención concentrados en una sola posición?
2. ¿Por qué escalamos por \sqrt{d_k}?
3. ¿Qué cambia al introducir máscara causal?